<font size="6">IMPORTING</font>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import scipy

import glob

In [2]:
users = pd.read_csv('fatal_motor.csv')

#ALCOHOL
try:
    # Read the CSV file
    alc = pd.read_csv('alc-sev.csv', encoding='utf-16', delimiter='\t')
except pd.errors.ParserError as e:
    print("Warning: Some lines were skipped due to parsing errors.")
    print(e)
    
    
#DAY
try:
    # Read the CSV file
    day_of = pd.read_csv('day-sev.csv', encoding='utf-16', delimiter='\t')
except pd.errors.ParserError as e:
    print("Warning: Some lines were skipped due to parsing errors.")
    print(e)
 

#AGES
try:
    # Read the CSV file
    mid_age = pd.read_csv('65To75.csv', encoding='utf-16', delimiter='\t')
except pd.errors.ParserError as e:
    print("Warning: Some lines were skipped due to parsing errors.")
    print(e)
    
try:
    # Read the CSV file
    old_age = pd.read_csv('75UP.csv', encoding='utf-16', delimiter='\t')
except pd.errors.ParserError as e:
    print("Warning: Some lines were skipped due to parsing errors.")
    print(e)
    
try:
    # Read the CSV file
    teen_age = pd.read_csv('TeenCrash.csv', encoding='utf-16', delimiter='\t')
except pd.errors.ParserError as e:
    print("Warning: Some lines were skipped due to parsing errors.")
    print(e)

    
#GENDER AND AGE
try:
    # Read the CSV file
    gen_age = pd.read_csv('GenAge.csv', encoding='utf-16', delimiter='\t')
except pd.errors.ParserError as e:
    print("Warning: Some lines were skipped due to parsing errors.")
    print(e)

In [3]:
alc.head()
users.head()
day_of.head()
mid_age.head()
old_age.head()
teen_age.head()
gen_age.head()

,Unnamed: 0,Unnamed: 1,1,2,3,4,5,6,7,8,...,103,104,106,107,108,109,110,111,116,Grand Total
0,NaN,"83,348",3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2,1,NaN,NaN,"83,738"
1,FEMALE,"1,695","4,310","2,130","2,193","2,200","2,218","2,158","2,306","2,242",...,2.0,NaN,NaN,NaN,NaN,6,7,NaN,1.0,"358,944"
2,MALE,"2,145","4,406","2,230","2,186","2,267","2,351","2,410","2,146","2,333",...,NaN,1.0,3.0,2.0,1.0,13,9,1.0,NaN,"405,502"
3,UNKNOWN,"1,210",73,16,11,10,72,33,31,37,...,NaN,NaN,NaN,NaN,NaN,3,1,NaN,NaN,"2,509"
4,Grand Total,"88,398","8,792","4,376","4,390","4,477","4,641","4,601","4,483","4,612",...,2.0,1.0,3.0,2.0,1.0,24,18,1.0,1.0,"850,693"


In [4]:
#ignore for now

#merged_df = pd.concat([teen_age, old_age], axis=0, ignore_index=True)
#merged_df.head()